In [1]:
import numpy as np
import gdown

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, TensorDataset

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
def load_data_from_drive(file_url, output_path):
    # Extract file ID from URL
    file_id = file_url.split("/d/")[1].split("/view")[0]

    # Construct download link
    download_url = f"https://drive.google.com/uc?id={file_id}"

    # Download file
    gdown.download(download_url, output_path, quiet=False)

# Load the data from my drive
load_data_from_drive("https://drive.google.com/file/d/1IYVkJ16ZL1D-LKhPGzfl1eB2rjLKH4yC/view?usp=drive_link", "X_train_tabular.npy")
load_data_from_drive("https://drive.google.com/file/d/10uNMOfGVrgkAmgZdzbZ3DuBYmpqtYuW7/view?usp=drive_link", "y_train_tabular.npy")
load_data_from_drive("https://drive.google.com/file/d/1OBXuczL-OAg_OdME3HAzN9J3VdhqKGUm/view?usp=drive_link", "X_train_padding.npy")
load_data_from_drive("https://drive.google.com/file/d/107Qww9eGFVcu92P522bZimQhW0dSpbLs/view?usp=drive_link", "y_train_padding.npy")

In [2]:
# Load your input and output data
x_train = np.load("X_train_tabular.npy")
y_train = np.load("y_train_tabular.npy")
print(x_train.shape)
print(y_train.shape)

# print(x_train.min(axis = 0))
# print(x_train.max(axis = 0))
# print(x_train.var())
# print(x_train.std())


# Load your input and output data
# Motion, duration (seq_lenght), features ---> (203, 12246, 12)
x_train_padd = np.load("X_train_padding.npy")
y_train_padd = np.load("y_train_padding.npy")
print(x_train_padd.shape)
print(y_train_padd.shape)

# Convert the data to PyTorch tensors
x_train = torch.tensor(x_train).float()
y_train = torch.tensor(y_train).float()

batch_size = 25000
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
print(len(train_loader))

(4361620, 12)
(4361620, 14)
(203, 12246, 12)
(203, 12246, 14)
175


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

from CNN_1D import CNN_1D

# Instantiate your model
model = CNN_1D().to(device)
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(model)
for param in model.parameters():
  print(param.shape)

In [ ]:
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

loss_values = []
epoch_count = []

# Train the model for 10 epochs
for epoch in range(10):
    train_loss = 0
    for x_train, y_train in train_loader:

        x_train, y_train = x_train.to('cuda'), y_train.to('cuda') # single batch

        # Forward pass
        y_pred = model(x_train)

        # Zero the gradients
        optimizer.zero_grad()

        # Compute the loss
        loss = criterion(y_pred, y_train)
        train_loss += loss

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

    train_loss /= len(train_loader)

    # Print the loss
    print(f'Epoch: {epoch+1}, Loss: {train_loss:.4f}')
    loss_values.append(train_loss.cpu().detach().numpy())
    epoch_count.append(epoch)

In [ ]:
plt.figure(figsize = (8, 8))

plt.plot(epoch_count, loss_values, label = "training loss with CNN with selection block")

plt.title("Training loss curve")
plt.xlabel("Epochs")
plt.ylabel("Loss")

plt.legend()
plt.show()